In [2]:
import pandas as pd
df = pd.read_csv("data/fashion_data.csv")
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [3]:
df = df.drop(columns=["ProductID","NumImages"])

In [4]:
df.columns

Index(['ProductName', 'ProductBrand', 'Gender', 'Price (INR)', 'Description',
       'PrimaryColor'],
      dtype='object')

In [5]:
df = df.dropna()

In [6]:
df["Price (INR)"] = df["Price (INR)"].apply(lambda x : "price {} ".format(x))

In [7]:
df["str"] = df['ProductBrand'] +" "+ df['Gender'] +" "+ df['ProductName'] +" "+ df["PrimaryColor"]+  " " +  df["Price (INR)"]  + df["Description"]

In [8]:
df['str'][0]

'DKNY Unisex DKNY Unisex Black & Grey Printed Medium Trolley Bag  Black price 11745 Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer'

In [9]:
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
import os
from dotenv import load_dotenv
import requests

from langchain_google_genai import GoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import os
from dotenv import load_dotenv
from langchain import OpenAI
import openai
from langchain_community.vectorstores import Milvus
from langchain.chains import RetrievalQA 
from langchain.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate
from langchain.memory import PostgresChatMessageHistory

load_dotenv()

d:\anaconda\installed\envs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [10]:
df["str"][:10]

0    DKNY Unisex DKNY Unisex Black & Grey Printed M...
1    EthnoVogue Women EthnoVogue Women Beige & Grey...
2    SPYKAR Women SPYKAR Women Pink Alexa Super Ski...
3    Raymond Men Raymond Men Blue Self-Design Singl...
4    Parx Men Parx Men Brown & Off-White Slim Fit P...
5    SHOWOFF Men SHOWOFF Men Brown Solid Slim Fit R...
6    Parx Men Parx Men Blue Slim Fit Checked Casual...
7    SPYKAR Women SPYKAR Women Burgundy Alexa Super...
8    Parx Men Parx Men Brown Tapered Fit Solid Regu...
9    DKNY Unisex DKNY Unisex Black Large Trolley Ba...
Name: str, dtype: object

In [11]:
seperator = "\n ************ \n"
data = " "
for i in df["str"][:20]:
    # print(type(data) ,type(i) ,type(seperator) , )
    data = data + i + seperator


In [12]:
def get_docs(collection_name):
    # data = get_data(collection_name)
    text_splitter = CharacterTextSplitter(
        separator = "************",
        chunk_size=900,
        chunk_overlap=0
    )
    dataDocument = [Document(page_content=data, metadata={"namespace": collection_name})]
    docs = text_splitter.split_documents(dataDocument)
    return docs



def create_vectorDB(collection_name):
    docs = get_docs(collection_name)
    initialize_embeddings()
    global vector_db
    vector_db = Milvus.from_documents(
        docs,
        embeddings,
        collection_name=collection_name,
        connection_args={
            "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
            "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
            "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
            # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
            "secure": True,
        },
    )


def initialize_embeddings():
    try :
        global embeddings
        embeddings = HuggingFaceInferenceAPIEmbeddings(
            api_key=os.getenv("HUGGINGFACE_API_KEY"), model_name="sentence-transformers/all-MiniLM-l6-v2"
        )
    except Exception as e :
        print("probelm accured while creating embeddings \n{}".format(e))


In [13]:
create_vectorDB("similarity")

In [14]:
text_splitter = CharacterTextSplitter(
    separator = "************",
    chunk_size=900,
    chunk_overlap=0
)
dataDocument = [Document(page_content=data, metadata={"namespace": "simm"})]
docs = text_splitter.split_documents(dataDocument)

In [17]:
def load_vectorDB(collection_name):
    global vector_db
    vector_db = Milvus(
    embeddings,
    collection_name=collection_name,
    connection_args={
    "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
    "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
    "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
    # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
    "secure": True,
    },
)
    
load_vectorDB("similarity")


In [19]:
print(data)

 DKNY Unisex DKNY Unisex Black & Grey Printed Medium Trolley Bag  Black price 11745 Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer
 ************ 
EthnoVogue Women EthnoVogue Women Beige & Grey Made to Measure Custom Made Kurta Set with Jacket  Beige price 5810 Beige & Grey made to measure kurta with churidar and dupattaBeige made to measure calf length kurta, has a V-neck, three-quarter sleeves, lightly padded on bust, flared hem, concealed zip closureGrey solid made to measure churidar, drawstring closureGrey net sequined dupatta, has printed tapingWhat is Made to Measure?Customised Kurta Set according to your Bust and Length. So please

In [21]:
vector_db.similarity_search(query="black jeans" , k=10)

[Document(page_content='SPYKAR Women SPYKAR Women Pink Alexa Super Skinny Fit High-Rise Clean Look Stretchable Cropped Jeans  Pink price 899 Pink coloured wash 5-pocket high-rise cropped jeans, clean look, no fade, has a button and zip closure, and waistband with belt loops\n ************ \nRaymond Men Raymond Men Blue Self-Design Single-Breasted Bandhgala Suit Blue price 5599 Blue self-design bandhgala suitBlue self-design bandhgala blazer, has a mandarin collar, single breasted with full button placket, long sleeves, three pockets, an attached lining and a double-vented back hemBlue self-design mid-rise trousers, has a zip fly with a button and a hook-and-bar closure, four pockets, a waistband with belt loops', metadata={'namespace': 'similarity'}),
 Document(page_content='Parx Men Parx Men Brown & Off-White Slim Fit Printed Casual Shirt  White price 759 Brown and off-white printed casual shirt, has a spread collar, long sleeves, button placket,  curved hem, one patch pocket\n ******